In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from data_for_gan import data_for_gan
from tqdm import tqdm
from torchsummary import summary
from tgan.model import TGANModel

ModuleNotFoundError: No module named 'tgan'

In [3]:
dataset,y,df = data_for_gan()

[1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0]


In [4]:
dataset = torch.from_numpy(dataset.astype('float32'))
y = torch.from_numpy(y.astype('int32'))

In [5]:
df

,activity no.,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f38,f39,f40,f41,f42,f43,f44,f45,Action,Human
0,1.0,-222.737122,540.995422,2832.409912,-233.136169,288.886017,2808.759277,-226.604401,85.114182,2796.978516,...,-114.83284,2771.922119,-78.613121,-542.027649,2771.436035,-18.19173,-930.296204,2827.437988,1.0,1.0
1,1.0,-222.75412,541.178589,2830.108398,-231.847168,288.444183,2809.476074,-225.781021,84.746086,2797.4104,...,-115.326927,2772.170166,-77.628181,-544.114624,2772.243164,-16.370182,-935.252625,2825.971436,1.0,1.0
2,1.0,-221.253021,542.361633,2828.212402,-231.197784,286.976593,2810.302002,-225.112366,84.134377,2798.335205,...,-115.031052,2772.760986,-76.57428,-543.925232,2773.098633,-16.996519,-935.505066,2828.508057,1.0,1.0
3,1.0,-220.483627,542.407715,2827.968262,-230.249573,285.834198,2810.092773,-224.584686,83.829437,2798.306396,...,-114.668465,2772.653076,-75.057556,-544.48761,2775.745117,-16.88364,-937.05603,2833.645752,1.0,1.0
4,1.0,-221.115891,542.176758,2829.080566,-230.501373,285.660583,2810.690918,-224.820831,83.872169,2798.683105,...,-114.363319,2772.71167,-74.918259,-544.191223,2775.963623,-16.241182,-935.5224,2832.526611,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6445,215.0,-251.36673,615.119385,2651.102295,-213.644806,366.986511,2745.237793,-207.447266,156.703217,2786.439209,...,-62.636711,2768.670898,-117.46299,-495.449005,2783.337891,-110.195564,-921.249329,2863.574463,9.0,10.0
6446,215.0,-251.36673,615.119385,2651.102295,-213.644806,366.986511,2745.237793,-207.447266,156.703217,2786.439209,...,-62.636711,2768.670898,-117.46299,-495.449005,2783.337891,-110.195564,-921.249329,2863.574463,9.0,10.0
6447,215.0,-223.757309,636.501526,2682.291016,-198.897583,382.04184,2759.463623,-202.598618,164.404648,2791.465576,...,-62.891205,2767.439941,-114.05262,-498.39682,2785.745605,-107.40934,-923.218323,2864.661865,9.0,10.0
6448,215.0,-223.757309,636.501526,2682.291016,-198.897583,382.04184,2759.463623,-202.598618,164.404648,2791.465576,...,-62.891205,2767.439941,-114.05262,-498.39682,2785.745605,-107.40934,-923.218323,2864.661865,9.0,10.0


In [4]:
dataset.size()

torch.Size([215, 3, 15, 30])

In [5]:
maxJoint = torch.max(dataset)
minJoint = torch.min(dataset)

In [6]:
dataset = (dataset - minJoint) / (maxJoint - minJoint)
dataset = dataset - 0.5

In [7]:
class CustomDataset(Dataset):
    def __init__(self,imgs,labels):
        self.imgs = imgs
        self.labels = labels
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.imgs[idx], self.labels[idx]

In [8]:
dataset = CustomDataset(dataset, y)

In [9]:
dataloader = DataLoader(
    dataset=dataset,
    batch_size=43,
    shuffle=True,
)

In [10]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Sequential(
            nn.Conv2d(4, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(negative_slope=0.2, inplace=True))
        self.embed = nn.Embedding(10 , 15 * 30)
        self.hiddenLayers = nn.Sequential(
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.BatchNorm2d(128),
            nn.Conv2d(128,256,3,1,1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(negative_slope=0.2, inplace=True)    
        )
        self.fc = nn.Linear(115200, 128)
        self.output = nn.Sequential(
            nn.Linear(128,1),
            nn.Sigmoid()
        )
    def forward(self, x, label):
        label_embed = self.embed(label)
        label_embed = label_embed.view(label_embed.size(0),1,15,30)
        finalInput = torch.cat([x, label_embed], dim=1)
        finalInput = self.input(finalInput)
        finalInput = self.hiddenLayers(finalInput)
        finalInput = finalInput.view(finalInput.size(0),-1)
        finalInput = self.fc(finalInput)
        finalInput = self.output(finalInput)
        return finalInput

In [11]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(10, 50)
        self.input = nn.Sequential(
            nn.ConvTranspose2d(150, 512, 4, 1, 0),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.hiddenlayers = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(), 
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),  
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),         
            nn.ConvTranspose2d(64, 16, 4 , 2, 1) 
        )
        self.layer = nn.Linear(65536, 1350)
        self.output = nn.Tanh()
    def forward(self,x, label):
        labelEmb = self.embed(label)
        labelEmb = labelEmb.view(labelEmb.size(0), 50, 1, 1)
        finalInput = torch.cat([x, labelEmb], dim = 1)
        finalInput = self.input(finalInput)
        finalInput = self.hiddenlayers(finalInput)
        finalInput = finalInput.view(finalInput.size(0),-1)
        finalInput = self.layer(finalInput)
        
        finalInput = finalInput.view(finalInput.size(0), 3, 15, 30)
        return self.output(finalInput)

In [23]:
print(Discriminator())

Discriminator(
  (input): Sequential(
    (0): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (embed): Embedding(10, 450)
  (hiddenLayers): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (fc): Linear(in_features=115200, out_features=128, bias=True)
  (output): Sequential(
    (0): Linear(in_features=128, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
device

device(type='cuda')

In [14]:
'''
Models
'''
generator = Generator().to(device)
discriminator = Discriminator().to(device)

In [15]:
'''
Losses
'''
adversarial_loss = nn.BCELoss()
adversarial_loss = adversarial_loss.cuda()

In [16]:
'''
Optimizers
'''
optimizer_G = torch.optim.Adam(generator.parameters(),lr=0.00001, betas=(0.5, 0.999) )
optimizer_D = torch.optim.Adam(discriminator.parameters(),lr=0.00001, betas=(0.5, 0.999))

In [17]:
cuda = torch.cuda.is_available()

In [18]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

epochs = 2000

In [19]:
for epoch in tqdm(range(epochs)):
    gLoss = -1
    dLoss = -1
    f1score = float("inf")
    for index, (data, label) in enumerate(dataloader):

        valid = Variable(Tensor(data.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(data.size(0), 1).fill_(0.0), requires_grad=False)

        label = label.to(device)

        real_imgs = Variable(data.type(Tensor))
        optimizer_G.zero_grad()

        z = Variable(Tensor(np.random.normal(0, 1, (data.shape[0], 100,1,1))))

        gen_imgs = generator(z,label)

        gen_imgs = gen_imgs.to(device)

        g_loss = adversarial_loss(discriminator(gen_imgs,label), valid)
        g_loss.backward()
        optimizer_G.step()

        optimizer_D.zero_grad()

        real_loss = adversarial_loss(discriminator(real_imgs,label), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach(),label), fake)

        d_loss = (real_loss + fake_loss)

        d_loss.backward()

        optimizer_D.step()

        gLoss = g_loss.item()
        dLoss = d_loss.item()

        f1Temp = -(gLoss * dLoss) / (gLoss + dLoss)
        if f1Temp < f1score:
            f1score = f1Temp
            # torch.save(generator.state_dict(),'Models/generator.pth')
    if (epoch+1)%50 == 0:
        print('gLoss ==>',round(gLoss,4))
        print("dLoss ==>",round(dLoss,4))


  2%|▎         | 50/2000 [00:27<16:12,  2.01it/s] 

gLoss ==> 0.7957
dLoss ==> 1.2767


  5%|▌         | 100/2000 [00:52<15:52,  1.99it/s]

gLoss ==> 1.9914
dLoss ==> 0.4174


  8%|▊         | 150/2000 [01:17<15:26,  2.00it/s]

...

gLoss ==> 3.0017
dLoss ==> 0.5676


 98%|█████████▊| 1950/2000 [16:23<00:25,  1.99it/s]

gLoss ==> 4.5211
dLoss ==> 0.8446


100%|██████████| 2000/2000 [16:48<00:00,  1.98it/s]

gLoss ==> 2.8547
dLoss ==> 0.8829


In [20]:
images = generator(torch.randn(1, 100 , 1 , 1).to(device), torch.tensor([[1]]).to(device))

In [21]:
images = images + 0.5
images = images * (maxJoint - minJoint)
images = images + minJoint


In [22]:
images

tensor([[[[ -142.6859,   -53.8264,   -90.2791,  ...,   -49.0355,
               9.2167,   -16.2069],
          [ -219.1793,  -121.1600,  -113.7405,  ...,  -101.0005,
             -48.0618,   -38.3032],
          [ -154.7822,   -87.2634,   -46.0358,  ...,   -96.2675,
             -37.9850,  -105.6514],
          ...,
          [   -8.6609,   -36.5859,     5.3986,  ...,    23.4196,
             -37.5100,   100.0842],
          [ -132.7388,   -35.0118,    29.1183,  ...,    -9.4705,
              22.2977,   -21.3118],
          [  133.1549,   107.9680,    41.0142,  ...,    87.5249,
               4.0879,   -26.9163]],

         [[  595.9054,   645.4562,   620.5333,  ...,   569.2972,
             582.2992,   601.0936],
          [  290.1754,   324.6591,   372.0774,  ...,   341.7295,
             408.1864,   311.1992],
          [  193.1750,   120.2783,   127.5544,  ...,   122.0002,
             182.2882,   166.5129],
          ...,
          [ -124.2042,   -84.6405,  -100.6724,  ...,   -61.